In [1]:
import pandas
import os
import graphviz

In [2]:
traindf=pandas.read_csv('train.csv')

testdf=pandas.merge(pandas.read_csv( 'test.csv'),
                    pandas.read_csv('gender_submission.csv'),
                    on="PassengerId")

In [3]:
traindf.var()

PassengerId    66231.000000
Survived           0.236772
Pclass             0.699015
Age              211.019125
SibSp              1.216043
Parch              0.649728
Fare            2469.436846
dtype: float64

In [4]:
for k in traindf.keys():
    print('{0}: {1}'.format(k, len(traindf[k].unique())))

PassengerId: 891
Survived: 2
Pclass: 3
Name: 891
Sex: 2
Age: 89
SibSp: 7
Parch: 7
Ticket: 681
Fare: 248
Cabin: 148
Embarked: 4


In [5]:
for k in traindf.keys():
    if len(traindf[k].unique())<=15:
        print(k)

Survived
Pclass
Sex
SibSp
Parch
Embarked


In [6]:
import math

def forAge(row):
    if row['Age'] < 10:
        return '<10'
    elif math.isnan(row['Age']):
        return "nan"
    else:
        dec = str(int(row['Age']/10))
        return "{0}0's".format(dec)

decade=traindf.apply(forAge, axis=1)
print("decade: {1}".format(k, len(decade.unique())))

decade: 10


In [7]:
import math
def pretreat(df):
    # create a df with discrete variables (len<10)
    lvd=[k for k in df.keys() if len(df[k].unique())<=15]
    res=df[lvd].copy()
    
    forAge = lambda row: "{0}0th".format(int(row['Age']/10)) if not math.isnan(row['Age']) else str("nan")
    res['decade']=df.apply(forAge, axis=1)
   
    return res

In [8]:
ptraindf=pretreat(traindf)
ptestdf=pretreat(testdf)

In [9]:
ptraindf.to_csv( 'post_train.csv', index=False)
ptestdf.to_csv( 'post_test.csv', index=False)

In [10]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [11]:
df = pandas.read_csv('post_train.csv')
for k in df.keys():
    print("{} : {}".format(k, df[k].unique()))

Survived : [0 1]
Pclass : [3 1 2]
Sex : ['male' 'female']
SibSp : [1 0 3 4 2 5 8]
Parch : [0 1 2 5 3 4 6]
Embarked : ['S' 'C' 'Q' nan]
decade : ['20th' '30th' nan '50th' '00th' '10th' '40th' '60th' '70th' '80th']


In [16]:
template=gum.BayesNet()
template.add(gum.RangeVariable("Survived", "Survived",0,1))
template.add(gum.RangeVariable("Pclass", "Pclass",1,3))
template.add(gum.LabelizedVariable("Sex", "Sex",0).addLabel("female").addLabel("male"))
template.add(gum.RangeVariable("SibSp", "SibSp",0,8))
template.add(gum.RangeVariable("Parch", "Parch",0,9))
template.add(gum.LabelizedVariable("Embarked", "Embarked",0).addLabel('').addLabel('C').addLabel('Q').addLabel('S'))
template.add(gum.LabelizedVariable("Decade", "Calculated decade", 10).addLabel(''))
gnb.showBN(template)

InvocationException: GraphViz's executables not found

In [15]:

os.environ["PATH"] += os.pathsep + 'C:\\Users\\ahn92\Anaconda3\\Lib\\site-packages\\graphviz'